In [38]:
import pandas as pd
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import Lasso
import numpy as np

In [59]:
data = pd.read_csv("data.csv")

In [60]:
data

,sex,location,loss
0,M,"""Out""",50
1,F,"""Out""",150
2,F,"""Out""",0
3,M,"""In""",200
4,M,"""In""",300
5,F,"""Out""",0
6,F,"""In""",0
7,M,"""Out""",0
8,F,"""In""",120
9,M,"""In""",100


In [61]:
X = data.drop('loss', axis=1)
#X = pd.get_dummies(X, drop_first=True)
y = data.loss
y = np.log1p(y)

In [65]:
X

,sex,location
0,M,"""Out"""
1,F,"""Out"""
2,F,"""Out"""
3,M,"""In"""
4,M,"""In"""
5,F,"""Out"""
6,F,"""In"""
7,M,"""Out"""
8,F,"""In"""
9,M,"""In"""


In [52]:
model = BaggingRegressor(LassoCV(random_state=42), oob_score=True)

In [53]:
model.fit(X,y)

BaggingRegressor(base_estimator=LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=42, selection='cyclic', tol=0.0001,
    verbose=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=True,
         random_state=None, verbose=0, warm_start=False)

In [54]:
X

,sex_F,sex_M,"location_ ""In""","location_ ""Out"""
0,0.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0
3,0.0,1.0,1.0,0.0
4,0.0,1.0,1.0,0.0
5,1.0,0.0,0.0,1.0
6,1.0,0.0,1.0,0.0
7,0.0,1.0,0.0,1.0
8,1.0,0.0,1.0,0.0
9,0.0,1.0,1.0,0.0


In [41]:
pd.DataFrame({'pred': model.oob_prediction_, 'loss': y})

,loss,pred
0,3.931826,2.754990
1,5.017280,0.262527
2,0.000000,1.540112
3,5.303305,4.113390
4,5.707110,4.002806
5,0.000000,1.003637
6,0.000000,3.953225
7,0.000000,4.244465
8,4.795791,2.166851
9,4.615121,4.732892


In [46]:
X_test = pd.DataFrame({'sex': ['M','M','F','F'], 'location': ['"In"','"Out"','"In"','"Out"']})
X_test = pd.get_dummies(X_test)

In [47]:
X_test["loss"] = np.exp(model.predict(X_test)) - 1

In [48]:
X_test["premium"] = X_test["loss"] / 0.7

In [49]:
X_test

,"location_""In""","location_""Out""",sex_F,sex_M,loss,premium
0,1.0,0.0,0.0,1.0,2.192854,3.132648
1,0.0,1.0,0.0,1.0,15.651550,22.359358
2,1.0,0.0,1.0,0.0,12.724131,18.177330
3,0.0,1.0,1.0,0.0,70.574858,100.821226
